In [ ]:
pip install gliner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cac

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_excel('/content/DocTopMisinformationPolicy (1).xlsx')

In [ ]:
data.head()

,Unnamed: 0,Document,Llama2
0,0,enhancing national security countering malign ...,"['Countering Russian Influence Abroad', '', ''..."
1,1,ukraine support security enhancement act outli...,"['Countering Russian Influence Abroad', '', ''..."
2,2,safeguarding election deepfake manipulation ac...,"['Election Integrity and Security', '', '', ''..."
3,3,strengthening global democracy act address wea...,"['Countering Chinese Influence Operations', ''..."
4,4,ethiopia democracy human right support strateg...,"['""Combating Misinformation and Disinformation..."


In [ ]:
data=data[['Document','Llama2']]

In [ ]:
data.head()

,Document,Llama2
0,enhancing national security countering malign ...,"['Countering Russian Influence Abroad', '', ''..."
1,ukraine support security enhancement act outli...,"['Countering Russian Influence Abroad', '', ''..."
2,safeguarding election deepfake manipulation ac...,"['Election Integrity and Security', '', '', ''..."
3,strengthening global democracy act address wea...,"['Countering Chinese Influence Operations', ''..."
4,ethiopia democracy human right support strateg...,"['""Combating Misinformation and Disinformation..."


In [ ]:
l1=[]
for x in data['Llama2']:
  y=x.split(',')[0]
  y=y.replace('[','')
  y=y.replace('\'','')
  y=y.replace('"','')
  l1.append(y)

In [ ]:
data['Topic']=l1

In [ ]:
data['Topic'].unique()

array(['Countering Russian Influence Abroad',
       'Election Integrity and Security',
       'Countering Chinese Influence Operations',
       'Combating Misinformation and Disinformation in National Security and Census',
       'Combating Misinformation in Public Health: The Role of Social Science and Scientific Research',
       'Combating Domestic Terrorism',
       'Cybersecurity Threats and Evolving Challenges'], dtype=object)

In [ ]:
from gliner import GLiNER

In [ ]:
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/781M [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

In [ ]:
df1=pd.DataFrame()
for document,summary in zip(list(data['Topic']),list(data['Document'])):
  text = summary
  labels = ["Topic"]
  entities = model.predict_entities(text, labels, threshold=0.5)
  dict1={}
  for entity in entities:
    if entity['label'] not in dict1.keys():
      dict1[entity['label']]=entity['text']
    else:
      dict1[entity['label']]=dict1[entity['label']]+','+entity['text']
  for x in dict1.keys():
    dict1[x]=dict1[x].split(',')
  if dict1!={}:
    max_length = max(len(values) for values in dict1.values())
    for key, values in dict1.items():
      dict1[key] = values + [None] * (max_length - len(values))
    df = pd.DataFrame(dict1)
  df['Topic1']=document
  df1=pd.concat([df1,df])

In [ ]:
df1.head()

,Topic,Topic1
0,countering malign influence,Countering Russian Influence Abroad
1,cybersecurity information literacy,Countering Russian Influence Abroad
0,russian aggression,Countering Russian Influence Abroad
1,russian aggression,Countering Russian Influence Abroad
0,deepfake content,Election Integrity and Security


In [ ]:
df1.to_excel('NER.xlsx')